In [1]:
from txtai.embeddings import Embeddings

/opt/anaconda3/envs/portfolioAi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_set_version=1.1
data_set_file_name = f"dataset_v{data_set_version}.json"

In [3]:
import json

def process_json_dataset(filepath):
    """
    Reads a JSON file, extracts data, and transforms it into a list of dictionaries.

    Args:
        filepath (str): The path to the JSON file.

    Returns:
        list: A list of dictionaries, where each dictionary has "output" and "text_input" keys.
    """
    try:
      with open(filepath, 'r') as f:
          data = json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

    index_data = []
    pair_counter = 0
    all_ids = set()
    for category, chunks in data.items():
        print(f"Processing category: {category}")
        for item in chunks:
            chunk_id_base = item['id']
            answer_chunk = item['text']
            chunk_metadata = item['metadata']

            for i, query in enumerate(item['questions']):
                pair_id = f"{chunk_id_base}_q{i}" # Generate a unique ID for the pair
                pair_counter += 1

                # Data object stores the question, the answer, and metadata
                data_object = {
                    "text": query, # This is what txtai will embed by default if 'text' key exists
                    "answer": answer_chunk,
                    "category": category,
                    "metadata": chunk_metadata
                }

                # Append tuple: (unique_pair_id, data_object_to_index, optional_tags)
                # We use the 'question' field for embedding similarity.
                index_data.append((pair_id, data_object, None)) # Pass metadata as tags

    print(f"Prepared {len(index_data)} items for indexing.")
    print(index_data)
    return index_data



filepath = f"./mounts/data/{data_set_file_name}"

index_data = process_json_dataset(filepath)

Processing category: general
Processing category: tech
Processing category: portfolio_AI
Processing category: xcuisite
Prepared 229 items for indexing.
[('gen_001_q0', {'text': 'Who are you?', 'answer': "Hey there, I'm Zi Shen!", 'category': 'general', 'metadata': {'topic': 'identity', 'subtopic': 'greeting'}}, None), ('gen_002_q0', {'text': 'Tell me more about yourself', 'answer': 'I am Zi Shen, a Year 3 student studying Math and Computer Science at Nanyang Technological University (NTU). My current focus is on backend development and integrating AI into projects.', 'category': 'general', 'metadata': {'topic': 'identity', 'subtopic': 'background_current_focus'}}, None), ('gen_003_q0', {'text': 'What is your background or experience?', 'answer': 'I have a background in Math and Computer Science from NTU. My experience comes from academic projects, internships, and personal endeavors showcased in this portfolio.', 'category': 'general', 'metadata': {'topic': 'identity', 'subtopic': 'exp

In [4]:
import os

def indexing(filePath, save=False):

    # Create embeddings in dex with content enabled. The default behavior is to only store indexed vectors.
    embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2", "content": True})

    # Map question to text and store content
    embeddings.index(index_data)

    if(save):
        if not os.path.exists(index_path_qa):
            os.makedirs(index_path_qa)
        print(f"Saving QA index to {index_path_qa}...")
        embeddings.save(index_path_qa)
        print("QA Index saved.")

    try:
        # --- !!! ADD THIS CHECK !!! ---
        index_count_after = embeddings.count()
        print(f"[indexing function] Count immediately after .index() call: {index_count_after}")
        if index_count_after == 0 and len(index_data) > 0:
            print("[indexing function] CRITICAL: Index count is 0 but data was provided!")
        # --- END CHECK ---

        print("Indexing completed successfully in memory (according to try block).")

    except Exception as e:
        print(f"Error during embeddings.index(): {e}")
        return None # Return None if indexing failed

    return embeddings

index_path_qa = "./mounts/index"
embeddings = indexing(index_path_qa,save=True)

Saving QA index to ./mounts/index...
QA Index saved.
[indexing function] Count immediately after .index() call: 229
Indexing completed successfully in memory (according to try block).


In [ ]:

def find_answer(user_query):
    embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2", "content": True})
    index_path_qa = "./mounts/index"
    embeddings.load(index_path_qa) # Load the index from the specified path
    query_sql = f"SELECT text, answer, score FROM txtai WHERE similar('{user_query}') LIMIT 3"
    results = embeddings.search(query_sql)
    return results

def print_ans(search_results):
    if search_results:
        for result in search_results:
            print(result)
            print("-" * 10)
    else:
        print("  No similar questions found in the index.")
# Example Usage:
user_question = "Tell me about your xcuisite website"
search_results = find_answer(user_question)
print_ans(search_results)



{'text': 'tell me about xcuisite website', 'answer': "The XCuisite website is an e-commerce platform designed for selling doughnuts. A live demo is available at www.xcuisite.store, or you can click the link in the 'Projects' section.", 'score': 0.9506585597991943}
----------
{'text': 'tell me more about xcuisite website', 'answer': "The XCuisite Website project's technical goal was to deepen understanding of a full-stack website's flow, covering frontend/backend communication, error handling, authN/authZ, database queries, payment processing, scalability, and containerized cloud deployment with SSL and a custom domain.", 'score': 0.9222586750984192}
----------
{'text': 'can you show me xcuisite website?', 'answer': "The XCuisite website is an e-commerce platform designed for selling doughnuts. A live demo is available at www.xcuisite.store, or you can click the link in the 'Projects' section.", 'score': 0.8420232534408569}
----------
